In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,Dropout,Activation
from torch.utils.data import TensorDataset, DataLoader
from tensorflow.python.keras import metrics
from tensorflow.python import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from imblearn.over_sampling import SMOTE
import math
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support as sk
from sklearn.metrics import f1_score ## F1 Score 구하기
from sklearn.metrics import accuracy_score
import pandas as pd
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
os.chdir("..")
path=os.getcwd()

In [3]:
df_final=pd.read_csv(path+'\\data\\5.서울초_df_final.csv',encoding="cp949")

In [4]:
name='서울초'

In [5]:
if name=='서울초':   
    df_final=pd.read_csv(path+'\\data\\5.서울초_df_final.csv',encoding="cp949")
    
if name=='제주초':   
    df_final=pd.read_csv(path+'\\data\\5.제주초_df_final.csv',encoding="cp949")

In [6]:
df_final

,ID,collect_datetime,gender,grade,height,weight,step count,burned calorie,eat calorie,sleep time
0,#AAGQKY,2022-07-14,2,4,148.0,45.15,2072.0,53.55,1939.050,480.0
1,#AAGQKY,2022-07-15,2,4,148.0,45.15,2072.0,53.55,1850.400,480.0
2,#AAGQKY,2022-07-16,2,4,148.0,45.15,2072.0,53.55,2567.025,480.0
3,#AAGQKY,2022-07-17,2,4,148.0,45.15,2072.0,53.55,2567.025,480.0
4,#AAGQKY,2022-07-18,2,4,148.0,45.15,2072.0,53.55,2066.400,480.0
...,...,...,...,...,...,...,...,...,...,...
47315,#ZWOHRX,2023-01-07,2,6,156.9,47.50,4929.0,53.55,2547.450,468.0
47316,#ZWOHRX,2023-01-08,2,6,156.9,47.50,4929.0,53.55,2547.450,468.0
47317,#ZWOHRX,2023-01-09,2,6,156.9,47.50,4929.0,53.55,2547.450,468.0
47318,#ZWOHRX,2023-01-10,2,6,156.9,47.50,4929.0,53.55,2547.450,468.0


## ID 라벨 인코딩

In [7]:
from Datapreprocessing.id_labelencoding import LabelEncoderWrapper 

df_encoder = LabelEncoderWrapper(df_final, 'ID')
df_encoder.fit()
df_encoder.transform()

## m day n term

일별 데이터를 다루기 때문에 처음에 접근한 방법으로는 몸무게의 변동이 거의 없을 수 밖에 없었다.   
그래서 14일치 묶음으로 1일 윈도우로 이동하면서 몸무게 변동의 유의미한 차이를 만들기 위한 데이터세트를 구축하였다.  
따라서 한 로우에 14일치 평균 그리고 다음 로우는 그 다음날부터 14일치 평균값이다. 

In [8]:
from Datapreprocessing.M_day_N_term import RollingAveragesCalculator
M_day=14
n_term=1
columns_to_average = ["ID",'height', 'weight', 'step count', 'burned calorie', 'eat calorie', 'sleep time']
calculator = RollingAveragesCalculator(df_final)
result_df = calculator.calculate_averages(columns_to_average, M_day, n_term)

In [9]:
name='{}day_{}term'.format(M_day,n_term)
print(name)

14day_1term


## 라벨링

라벨링은 임계값 몸무게 변동 차이 100g을 기준으로 이전 평균 몸무게 보다 증가하면 3, 유지면 2, 감소면 1로 라벨링하였다.    

In [10]:
from Datapreprocessing.labeling import WeightChangeLabeler

Threshhold=0.1 # 몸무게가 얼마나 차이나야 변했다고 정할 것 인지 정하는 임계값
labeler = WeightChangeLabeler(result_df, Threshhold)
labeled_df = labeler.label_weight_changes()
print(labeled_df)

        ID  height  weight  step count  burned calorie  eat calorie  \
0        0   148.0   45.15      2072.0           53.55  2300.576786   
1        0   148.0   45.15      2072.0           53.55  2345.432143   
2        0   148.0   45.15      2072.0           53.55  2396.619643   
3        0   148.0   45.15      2072.0           53.55  2396.619643   
4        0   148.0   45.15      2072.0           53.55  2396.619643   
...    ...     ...     ...         ...             ...          ...   
43935  259   156.9   47.50      4929.0           53.55  2733.846429   
43936  259   156.9   47.50      4929.0           53.55  2733.846429   
43937  259   156.9   47.50      4929.0           53.55  2743.489286   
43938  259   156.9   47.50      4929.0           53.55  2716.264286   
43939  259   156.9   47.50      4929.0           53.55  2703.664286   

       sleep time  Start_Day Label  
0           480.0          0     2  
1           480.0          1     2  
2           480.0          2     2  

In [11]:
labeled_df['Label']=labeled_df['Label'].astype(int)

## 데이터 생성

합성데이터를 생성  
합성 데이터는 Smote, Adasyn, CopulaGAN, CTGAN, Nbsynthetic(WGAN으로 생성)  
각 합성데이터를 이용하여 모델학습해서 성능 비교테스트  

### Smote

In [12]:
from Datapreprocessing.scaling_smote import DataScalerAndSampler
processor_smote = DataScalerAndSampler(labeled_df, method='smote')
processor_smote.scale_data()
processor_smote.apply_sampling()

X_over_1, y_over_1, X_1, y_1 = processor_smote.get_processed_data()
# 스케일링 된거 inverse
X_scaler_inverse=processor_smote.inverse_scale(X_1)
X_scaler_over_inverse=processor_smote.inverse_scale(X_over_1)
X_scaler_df= pd.DataFrame(X_scaler_inverse, columns=columns_to_average[1:])
X_scaler_over_df= pd.DataFrame(X_scaler_over_inverse, columns=columns_to_average[1:])
# 각 행의 모든 컬럼이 일치하는지 확인하고, 결과를 'check' 컬럼에 저장
X_scaler_over_df['Smote_check'] = X_scaler_over_df.ne(X_scaler_df).any(axis=1).astype(int)
X_scaler_over_df["Label"]=y_over_1
df_final_merge=labeled_df.iloc[:,:-2]
X_scaler_over_df_notSmote=X_scaler_over_df[X_scaler_over_df['Smote_check']==0].reset_index(drop=True)
X_scaler_over_df_Smote=X_scaler_over_df[X_scaler_over_df['Smote_check']==1].reset_index(drop=True)
X_scaler_over_df_notSmote['ID']=df_final_merge['ID']
X_scaler_over_df_Smote['ID']='999'
X_scaler_over_df_result = pd.concat([X_scaler_over_df_notSmote, X_scaler_over_df_Smote], axis=0)
X_scaler_over_df_result_smote=X_scaler_over_df_result.reset_index(drop=True)
X_scaler_over_df_result_smote.to_csv(path+f"\\Synthetic_data\\{name}_smote_생성데이터프레임.csv",encoding="cp949",index=False)

In [13]:
print('SMOTE 적용 전 레이블 값 분포: \n', pd.Series(X_scaler_over_df_result_smote['Label']).value_counts())


SMOTE 적용 전 레이블 값 분포: 
 Label
2    42277
3    11159
1    10504
Name: count, dtype: int64


### Adasyn

In [14]:
from Datapreprocessing.scaling_smote import DataScalerAndSampler
from imblearn.over_sampling import ADASYN
processor_adasyn = DataScalerAndSampler(labeled_df, method='adasyn')
processor_adasyn.scale_data()
processor_adasyn.apply_sampling()

X_over_1, y_over_1, X_1, y_1 = processor_adasyn.get_processed_data()

# 스케일링 된거 inverse
X_scaler_inverse=processor_adasyn.inverse_scale(X_1)
X_scaler_over_inverse=processor_adasyn.inverse_scale(X_over_1)
X_scaler_df= pd.DataFrame(X_scaler_inverse, columns=columns_to_average[1:])
X_scaler_over_df= pd.DataFrame(X_scaler_over_inverse, columns=columns_to_average[1:])
X_scaler_over_df['Smote_check'] = X_scaler_over_df.ne(X_scaler_df).any(axis=1).astype(int)
X_scaler_over_df["Label"]=y_over_1
df_final_merge=labeled_df.iloc[:,:-2]
X_scaler_over_df_notadasyn=X_scaler_over_df[X_scaler_over_df['Smote_check']==0].reset_index(drop=True)
X_scaler_over_df_Smote=X_scaler_over_df[X_scaler_over_df['Smote_check']==1].reset_index(drop=True)
X_scaler_over_df_notadasyn['ID']=df_final_merge['ID']
X_scaler_over_df_Smote['ID']='999'
X_scaler_over_df_result = pd.concat([X_scaler_over_df_notadasyn, X_scaler_over_df_Smote], axis=0)
X_scaler_over_df_result_adasyn=X_scaler_over_df_result.reset_index(drop=True)
X_scaler_over_df_result_adasyn.to_csv(path+f"\\Synthetic_data\\{name}_adasyn_생성데이터프레임.csv",encoding="cp949",index=False)

### CopulaGAN

In [15]:
# GAN을 통해 생성하는 것이기 때문에 GPU 사용 필요
import torch
torch.cuda.is_available()

True

In [16]:
from sdv.metadata import Metadata
from sdv.single_table import CopulaGANSynthesizer

labeled_1=labeled_df[labeled_df['Label']==1]
# 라벨 1 10000개 생성
metadata = Metadata.detect_from_dataframe(
    data=labeled_1,
    table_name='weight_change_1')

synthesizer1 = CopulaGANSynthesizer(metadata,epochs=500)
synthesizer1.fit(labeled_1)
synthetic_data_1 = synthesizer1.sample(num_rows=10000)


# 라벨 3 10000개 생성
labeled_3=labeled_df[labeled_df['Label']==3]
metadata = Metadata.detect_from_dataframe(
    data=labeled_3,
    table_name='weight_change_3')
synthesizer3 = CopulaGANSynthesizer(metadata,epochs=500)
synthesizer3.fit(labeled_3)
synthetic_data_3= synthesizer3.sample(num_rows=10000)

In [17]:
df_combined = pd.concat([synthetic_data_1, synthetic_data_3], axis=0)
df_combined=df_combined.reset_index(drop=True)
df_combined.to_csv(path+f'\\Synthetic_data\\{name}_copulagan_생성.csv',encoding='cp949',index=False)

In [18]:

origin_train = labeled_df
train_total = pd.concat([origin_train, df_combined])
train_total=train_total.reset_index(drop=True)
train_total.to_csv(path+f'\\Synthetic_data\\{name}_copulagan_생성데이터프레임.csv',encoding='cp949',index=False)


### CTGAN

In [19]:
# pip install ctgan

In [20]:
labeled_df

,ID,height,weight,step count,burned calorie,eat calorie,sleep time,Start_Day,Label
0,0,148.0,45.15,2072.0,53.55,2300.576786,480.0,0,2
1,0,148.0,45.15,2072.0,53.55,2345.432143,480.0,1,2
2,0,148.0,45.15,2072.0,53.55,2396.619643,480.0,2,2
3,0,148.0,45.15,2072.0,53.55,2396.619643,480.0,3,2
4,0,148.0,45.15,2072.0,53.55,2396.619643,480.0,4,2
...,...,...,...,...,...,...,...,...,...
43935,259,156.9,47.50,4929.0,53.55,2733.846429,468.0,164,2
43936,259,156.9,47.50,4929.0,53.55,2733.846429,468.0,165,2
43937,259,156.9,47.50,4929.0,53.55,2743.489286,468.0,166,2
43938,259,156.9,47.50,4929.0,53.55,2716.264286,468.0,167,2


In [21]:
# CTGAN 생성

import pandas as pd
import numpy as np
from ctgan import CTGAN
from tqdm import tqdm

# 불필요한 컬럼 제거 
labeled_df_CTGAN = labeled_df.drop(['ID', 'Start_Day'], axis=1, errors='ignore')
ctgan = CTGAN(batch_size=100, epochs=100, verbose=True)
all_synthetic_data = pd.DataFrame()
label_types=labeled_df_CTGAN['Label'].unique()

for label_type in tqdm(label_types):
   if label_type==2:
      continue
 # 해당 label_type에 대한 서브셋 생성
   subset = labeled_df_CTGAN[labeled_df_CTGAN["Label"] == label_type]
   ctgan.fit(subset)
   n_generated_data = 10000
   generated_df = ctgan.sample(n_generated_data)
   all_synthetic_data = pd.concat([all_synthetic_data, generated_df], ignore_index=True)


Gen. (-1.90) | Discrim. (-0.16): 100%|██████████| 100/100 [00:18<00:00,  5.30it/s]
Gen. (-1.46) | Discrim. (0.21): 100%|██████████| 100/100 [00:08<00:00, 11.16it/s]
100%|██████████| 3/3 [00:30<00:00, 10.03s/it]


In [22]:
all_synthetic_data.to_csv(path+f'\\Synthetic_data\\{name}_ctgan_생성.csv',encoding='cp949')
origin_train = labeled_df
train_total = pd.concat([origin_train, all_synthetic_data])
train_total=train_total.reset_index(drop=True)
train_total.to_csv(path+f'\\Synthetic_data\\{name}_ctgan_생성데이터프레임.csv',encoding='cp949',index=False)


### nbsynthetic

In [44]:
# 불필요한 컬럼 제거 (예시)
labeled_df_nbsynthetic = labeled_df.drop(['ID', 'Start_Day'], axis=1, errors='ignore')
all_synthetic_data = pd.DataFrame()

In [45]:
# nbsynthetic
from nbsynthetic.vgan import GAN
from nbsynthetic.synthetic import synthetic_data
import numpy as np
label_types=labeled_df_nbsynthetic['Label'].unique()

for label_type in tqdm(label_types):
    if label_type == 2:
        continue
    subset = labeled_df_nbsynthetic[labeled_df_nbsynthetic["Label"] == label_type]
    subset = subset + np.random.normal(0, 1e-6, subset.shape)  # 아주 작은 노이즈 추가
    samples = 10000 #3000
    newdf = synthetic_data(GAN, subset, samples=samples,epochs=30)
    newdf['Label'] = np.rint(newdf['Label']).astype(int)
    all_synthetic_data = pd.concat([all_synthetic_data, newdf], ignore_index=True)
  
    
all_synthetic_data.to_csv(path+f'\\Synthetic_data\\{name}_nbsynthetic_생성.csv',encoding='cp949',index=False)

  0%|          | 0/3 [00:00<?, ?it/s]

1/1 [==============================] - 0s 79ms/step


1/1 [==============================] - 0s 25ms/step


1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 21ms/step


...

1/1 [==============================] - 0s 40ms/step


1/1 [==============================] - 0s 23ms/step


Epoch (30/30) | D. loss: 0.33 | G. loss: 0.71 |: 100%|##########| 22/22 [00:03<00:00,  6.16it/s]


313/313 [==============================] - 2s 5ms/step


100%|██████████| 3/3 [05:25<00:00, 108.61s/it]


In [46]:
origin_train = labeled_df

train_total = pd.concat([origin_train, all_synthetic_data])
train_total=train_total.reset_index(drop=True)
train_total.to_csv(path+f'\\Synthetic_data\\{name}_nbsynthetic_생성데이터프레임.csv',encoding='cp949',index=False)


In [50]:
train_total['Label'].unique()

array([2, 3, 1])

In [48]:
train_total['Label']=train_total['Label'].astype(int)


In [49]:
path

'c:\\jupyter\\Explainable Healthcare framework\\Explainable_healthcareframework'